# Araba Fiyat Tahmini Projesi
## Bolum 1: Keşifsel Veri Analizi (EDA)

---

### Bu Notebook'ta Neler Yapacağız?
- Veri setini yükleme ve ilk bakış
- Temel istatistikler ve veri tipleri
- Eksik değer analizi
- Fiyat dağılımı analizi
- Kategorik değişken analizi
- Korelasyon analizi
- Önemli bulgular ve çıkarımlar

---

## 1. Kütüphanelerin Yüklenmesi

In [ ]:
# Temel kütüphaneler
import pandas as pd
import numpy as np

# Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns

# Türkçe karakter desteği ve stil ayarları
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Uyarıları kapat
import warnings
warnings.filterwarnings('ignore')

# Pandas ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Kütüphaneler başarıyla yüklendi!")

## 2. Veri Setinin Yüklenmesi

In [ ]:
# Veri setini yükle
df = pd.read_csv('../car_prices_tr.csv')

# İlk 5 satıra bakalım
print(f"Veri Seti Boyutu: {df.shape[0]:,} satır, {df.shape[1]} sütun")
print("\n" + "="*80)
df.head()

## 3. Veri Seti Genel Bilgileri

In [ ]:
# Sütun isimleri ve veri tipleri
print("SÜTUN BİLGİLERİ")
print("="*80)

info_df = pd.DataFrame({
    'Sütun Adı': df.columns,
    'Veri Tipi': df.dtypes.values,
    'Boş Olmayan': df.count().values,
    'Boş Sayısı': df.isnull().sum().values,
    'Boş Oranı (%)': (df.isnull().sum().values / len(df) * 100).round(2)
})

info_df

## 4. Fiyat Sütununun Temizlenmesi ve Analizi

Fiyat sütunu şu anda metin formatında: `"..000 TL"` → Bunu sayısal formata çevirmeliyiz.

In [ ]:
# Fiyat sütununu temizle
def temizle_fiyat(fiyat):
    """Fiyat metnini sayıya çevirir: '1.169.000 TL' -> 1169000"""
    if pd.isna(fiyat):
        return np.nan
    fiyat_str = str(fiyat)
    # TL ve boşlukları kaldır, noktaları kaldır
    fiyat_str = fiyat_str.replace(' TL', '').replace('.', '').strip()
    try:
        return int(fiyat_str)
    except:
        return np.nan

# Uygula
df['fiyat_temiz'] = df['fiyat(TRY)'].apply(temizle_fiyat)

# Sonucu kontrol et
print("FİYAT İSTATİSTİKLERİ")
print("="*80)
print(f"Minimum Fiyat: {df['fiyat_temiz'].min():,.0f} TL")
print(f"Maksimum Fiyat: {df['fiyat_temiz'].max():,.0f} TL")
print(f"Ortalama Fiyat: {df['fiyat_temiz'].mean():,.0f} TL")
print(f"Medyan Fiyat: {df['fiyat_temiz'].median():,.0f} TL")
print(f"Standart Sapma: {df['fiyat_temiz'].std():,.0f} TL")

In [ ]:
# Fiyat dağılımı grafiği
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sol: Histogram
axes[0].hist(df['fiyat_temiz'].dropna() / 1_000_000, bins=50, color='steelblue', edgecolor='white')
axes[0].set_xlabel('Fiyat (Milyon TL)', fontsize=12)
axes[0].set_ylabel('Araç Sayısı', fontsize=12)
axes[0].set_title('Fiyat Dağılımı', fontsize=14, fontweight='bold')
axes[0].axvline(df['fiyat_temiz'].median() / 1_000_000, color='red', linestyle='--', label='Medyan')
axes[0].legend()

# Sağ: Box Plot
axes[1].boxplot(df['fiyat_temiz'].dropna() / 1_000_000, vert=True)
axes[1].set_ylabel('Fiyat (Milyon TL)', fontsize=12)
axes[1].set_title('Fiyat Kutu Grafiği', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/figures/fiyat_dagilimi.png', dpi=150, bbox_inches='tight')
plt.show()

print("
YORUM: Fiyat dağılımı sağa çarpık (right-skewed). ")
print("Bu durum, log transformation yapılmasını gerektirir.")

## 5. Kilometre Sütununun Analizi

In [ ]:
# Kilometre sütununu temizle
def temizle_km(km):
    """Kilometre metnini sayıya çevirir: '120.000 km' -> 120000"""
    if pd.isna(km):
        return np.nan
    km_str = str(km)
    km_str = km_str.replace(' km', '').replace('.', '').strip()
    try:
        return int(km_str)
    except:
        return np.nan

df['km_temiz'] = df['kilometre(Km)'].apply(temizle_km)

print("KİLOMETRE İSTATİSTİKLERİ")
print("="*80)
print(f"Minimum KM: {df['km_temiz'].min():,.0f} km")
print(f"Maksimum KM: {df['km_temiz'].max():,.0f} km")
print(f"Ortalama KM: {df['km_temiz'].mean():,.0f} km")
print(f"Medyan KM: {df['km_temiz'].median():,.0f} km")

In [ ]:
# KM vs Fiyat scatter plot
plt.figure(figsize=(10, 6))

# Örnek olarak 2000 veri noktası al (daha hızlı çizim için)
sample_df = df.dropna(subset=['km_temiz', 'fiyat_temiz']).sample(min(2000, len(df)))

plt.scatter(sample_df['km_temiz'] / 1000, 
            sample_df['fiyat_temiz'] / 1_000_000, 
            alpha=0.5, c='steelblue', s=30)

plt.xlabel('Kilometre (bin km)', fontsize=12)
plt.ylabel('Fiyat (Milyon TL)', fontsize=12)
plt.title(' Kilometre vs Fiyat İlişkisi', fontsize=14, fontweight='bold')

plt.savefig('../outputs/figures/km_vs_fiyat.png', dpi=150, bbox_inches='tight')
plt.show()

print("
YORUM: Genel olarak düşük kilometreli araçlar daha yüksek fiyatlı.")
print("Ancak yüksek fiyatlı lüks araçlarda bu ilişki farklılaşabiliyor.")

## 6. Kategorik Değişken Analizi

In [ ]:
# Marka dağılımı
print("MARKA DAĞILIMI (İlk )")
print("="*80)
marka_dagilimi = df['marka'].value_counts().head()
print(marka_dagilimi)

# Grafik
plt.figure(figsize=(14, 5))
marka_dagilimi.plot(kind='bar', color='steelblue', edgecolor='white')
plt.xlabel('Marka', fontsize=12)
plt.ylabel('İlan Sayısı', fontsize=12)
plt.title(' En Çok İlan Verilen Markalar', fontsize=14, fontweight='bold')
plt.xticks(rotation=, ha='right')
plt.tight_layout()
plt.savefig('../outputs/figures/marka_dagilimi.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Yakıt türü dağılımı
print("YAKIT TÜRÜ DAĞILIMI")
print("="*80)
yakit_dagilimi = df['yakitTuru'].value_counts()
print(yakit_dagilimi)

# Pasta grafiği
plt.figure(figsize=(14, 5))
colors = sns.color_palette('husl', len(yakit_dagilimi))
plt.pie(yakit_dagilimi.values, labels=yakit_dagilimi.index, autopct='%.f%%', 
        colors=colors, startangle=0)
plt.title(' Yakıt Türü Dağılımı', fontsize=14, fontweight='bold')
plt.savefig('../outputs/figures/yakit_dagilimi.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Vites tipi dağılımı
print("VİTES TİPİ DAĞILIMI")
print("="*80)
vites_dagilimi = df['vitesTipi'].value_counts()
print(vites_dagilimi)

# Grafik
plt.figure(figsize=(14, 5))
vites_dagilimi.plot(kind='barh', color=['#ecc', '#db', '#ecc'])
plt.xlabel('İlan Sayısı', fontsize=12)
plt.ylabel('Vites Tipi', fontsize=12)
plt.title(' Vites Tipi Dağılımı', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../outputs/figures/vites_dagilimi.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Yıl Bazlı Analiz

In [ ]:
# Yıl dağılımı
print("YIL DAĞILIMI")
print("="*80)

# Yıl sütununu düzenle
df['yil_temiz'] = df['yıl'].astype(float).astype('Int')

# 000 sonrası araçlar
yil_dagilimi = df[df['yil_temiz'] >= 000]['yil_temiz'].value_counts().sort_index()

plt.figure(figsize=(14, 5))
yil_dagilimi.plot(kind='bar', color='steelblue', edgecolor='white')
plt.xlabel('Model Yılı', fontsize=12)
plt.ylabel('İlan Sayısı', fontsize=12)
plt.title(' Yıllara Göre Araç Dağılımı', fontsize=14, fontweight='bold')
plt.xticks(rotation=)
plt.tight_layout()
plt.savefig('../outputs/figures/yil_dagilimi.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Yıl ve Fiyat ilişkisi
plt.figure(figsize=(14, 5))

# Yıla göre ortalama fiyat
yil_fiyat = df[df['yil_temiz'] >= 00].groupby('yil_temiz')['fiyat_temiz'].median() / 1_000_000

yil_fiyat.plot(kind='line', marker='o', color='steelblue', linewidth=, markersize=)
plt.xlabel('Model Yılı', fontsize=12)
plt.ylabel('Medyan Fiyat (Milyon TL)', fontsize=12)
plt.title(' Yıla Göre Medyan Fiyat Değişimi', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.)
plt.tight_layout()
plt.savefig('../outputs/figures/yil_fiyat_iliskisi.png', dpi=150, bbox_inches='tight')
plt.show()

print("
YORUM: Beklendiği gibi yeni model araçlar daha yüksek fiyatlı.")

## 8. Eksik Değer Analizi

In [ ]:
# Eksik değer analizi
eksik_degerler = df.isnull().sum()
eksik_yuzde = (eksik_degerler / len(df) * 100).round(2)

eksik_df = pd.DataFrame({
    'Eksik Sayısı': eksik_degerler,
    'Eksik Oranı (%)': eksik_yuzde
}).sort_values('Eksik Oranı (%)', ascending=False)

eksik_df = eksik_df[eksik_df['Eksik Sayısı'] > 0]

print("EKSİK DEĞER ANALİZİ")
print("="*80)
print(eksik_df)

# Görselleştirme
if len(eksik_df) > 0:
    plt.figure(figsize=(10, 6))
    eksik_df['Eksik Oranı (%)'].plot(kind='barh', color='coral')
    plt.xlabel('Eksik Oran (%)', fontsize=12)
    plt.title(' Sütunlara Göre Eksik Değer Oranları', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../outputs/figures/eksik_degerler.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. İl Bazlı Fiyat Analizi

In [ ]:
# İllere göre ortalama fiyat
il_fiyat = df.groupby('il')['fiyat_temiz'].agg(['median', 'count'])
il_fiyat.columns = ['Medyan Fiyat', 'İlan Sayısı']
il_fiyat = il_fiyat[il_fiyat['İlan Sayısı'] >= 0]  # En az 0 ilan olan iller
il_fiyat = il_fiyat.sort_values('Medyan Fiyat', ascending=False).head()

print("EN PAHALI  İL (en az 0 ilan)")
print("="*80)
print(il_fiyat)

# Grafik
plt.figure(figsize=(14, 5))
plt.barh(il_fiyat.index, il_fiyat['Medyan Fiyat'] / 1_000_000, color='steelblue')
plt.xlabel('Medyan Fiyat (Milyon TL)', fontsize=12)
plt.ylabel('İl', fontsize=12)
plt.title(' İllere Göre Medyan Araç Fiyatları', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../outputs/figures/il_fiyat.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Hasar Durumu Analizi

In [ ]:
# Parça durumu analizi
parca_sutunlar = ['orjinal_parça_sayısı', 'lokal_boyalı_parça_sayısı', 
                  'boyalı_parça_sayısı', 'değişen_parça_sayısı', 
                  'durumu_belirtilmemiş_parça_sayısı']

parca_ozet = df[parca_sutunlar].describe()
print("PARÇA DURUMU İSTATİSTİKLERİ")
print("="*80)
print(parca_ozet)

# Ortalama değerleri göster
plt.figure(figsize=(10, 6))
ortalamalar = df[parca_sutunlar].mean()
ortalamalar.plot(kind='bar', color=['green', 'yellow', 'orange', 'red', 'gray'])
plt.ylabel('Ortalama Parça Sayısı', fontsize=12)
plt.title(' Ortalama Parça Durumu Dağılımı', fontsize=14, fontweight='bold')
plt.xticks(rotation=, ha='right')
plt.tight_layout()
plt.savefig('../outputs/figures/parca_durumu.png', dpi=150, bbox_inches='tight')
plt.show()

---

## ÖZET VE ÖNEMLİ BULGULAR

### Veri Seti Hakkında:
- **Toplam Kayıt:** , araç ilanı
- **Sütun Sayısı:**  özellik
- **Hedef Değişken:** Fiyat (TRY)

### Önemli Bulgular:

**Fiyat Dağılımı:** Sağa çarpık dağılım gösteriyor → **Log transformation gerekli**

**Kilometre:** Fiyat ile negatif korelasyon → **Önemli bir özellik**

**Yıl:** Yeni araçlar daha pahalı → **Araç yaşı türetilmeli**

**Kategorik Değişkenler:** Marka, yakıt tipi, vites tipi → **Encode edilmeli**

**Eksik Değerler:** Motor hacmi ve gücünde eksiklik var → **Imputation gerekli**

**Bölgesel Farklar:** İstanbul ve Ankara'da fiyatlar daha yüksek

### Sonraki Adımlar:
- `0_veri_temizleme.ipynb` → Veri ön işleme
- `0_model_egitimi.ipynb` → Model eğitimi ve optimizasyon

---

In [ ]:
# Temizlenmiş veriyi kaydet (bir sonraki notebook için)
df.to_csv('../outputs/data_eda_sonrasi.csv', index=False)
print("Veri kaydedildi: outputs/data_eda_sonrasi.csv")